In [1]:
import torch
from solver.solver import Solver
from models.cnn_transformer import get_resnet_transformer
from models.resnet101_3d_cnn import get_resnet101_3d
from dataset.loader import GestureDataset
from torch.utils.data import DataLoader
from models.utils import count_trainable_parameters
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import os
import numpy as np

In [2]:
annotations_train = "Annot_TrainList.txt"
annotations_test = "Annot_TestList.txt"

file_train = os.path.join(".", "IPN_Hand","annotations-20231128T085307Z-001", "annotations", annotations_train)
file_test = os.path.join(".", "IPN_Hand","annotations-20231128T085307Z-001", "annotations", annotations_test)
frame_folders = os.path.join(".", "IPN_Hand", "frames")


In [3]:
ds_train = GestureDataset(frame_folders, file_train)
ds_test = GestureDataset(frame_folders, file_test)

In [4]:
labes = np.array(ds_train.get_labels())[:, 4].astype(np.int32)
np.unique(labes)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [5]:
transform = transforms.Compose([
    transforms.ToTensor()  # Converts to FloatTensor and scales to [0.0, 1.0]
    # ... (other transforms, if necessary)
])

In [6]:
ds_train = GestureDataset(frame_folders, file_train, transform, sample_duration=15)
ds_test = GestureDataset(frame_folders, file_test, transform, sample_duration=15)

In [7]:
ds_train[0][0].shape

torch.Size([15, 3, 240, 320])

In [8]:
print(f"We have {len(ds_train)} samples for training")
print(f"We have {len(ds_test)} samples for testing")

We have 4039 samples for training
We have 1610 samples for testing


In [9]:
train_dataloader = DataLoader(ds_train, batch_size=8, shuffle=True)
test_dataloader = DataLoader(ds_test, batch_size=8)

In [10]:
# model = get_resnet_transformer(feature_size=6, nhead=2, nhid=32, nlayers=1, num_classes=13)
model = get_resnet101_3d()

In [11]:
count_trainable_parameters(model)

85236174

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [13]:
solver = Solver(model, train_dataloader, test_dataloader, criterion, optimizer, device)

In [14]:
num_epochs = 2
solver.train(num_epochs)

Batch 1 done!
Batch 2 done!
Batch 3 done!
Batch 4 done!
Batch 5 done!
Batch 6 done!
Batch 7 done!
Batch 8 done!
Batch 9 done!
Batch 10 done!
Batch 11 done!
Batch 12 done!
Batch 13 done!
Batch 14 done!
Batch 15 done!
Batch 16 done!
Batch 17 done!
Batch 18 done!
Batch 19 done!
Batch 20 done!
Batch 21 done!
Batch 22 done!
Batch 23 done!
Batch 24 done!
Batch 25 done!
Batch 26 done!
Batch 27 done!
Batch 28 done!
Batch 29 done!
Batch 30 done!
Batch 31 done!
Batch 32 done!
Batch 33 done!
Batch 34 done!
Batch 35 done!
Batch 36 done!
Batch 37 done!
Batch 38 done!
Batch 39 done!
Batch 40 done!
Batch 41 done!
Batch 42 done!
Batch 43 done!
Batch 44 done!
Batch 45 done!
Batch 46 done!
Batch 47 done!
Batch 48 done!
Batch 49 done!
Batch 50 done!
Batch 51 done!
Batch 52 done!
Batch 53 done!
Batch 54 done!
Batch 55 done!
Batch 56 done!
Batch 57 done!
Batch 58 done!
Batch 59 done!
Batch 60 done!
Batch 61 done!
Batch 62 done!
Batch 63 done!
Batch 64 done!
Batch 65 done!
Batch 66 done!
Batch 67 done!
Batc

In [15]:
solver.save("test.pt")

In [3]:
annotations_train = "Annot_TrainList.txt"
annotations_test = "Annot_TestList.txt"

file_train = os.path.join(".", "IPN_Hand","annotations-20231128T085307Z-001", "annotations", annotations_train)
file_test = os.path.join(".", "IPN_Hand","annotations-20231128T085307Z-001", "annotations", annotations_test)
frame_folders = os.path.join(".", "IPN_Hand", "hand_gestures.h5")

ds_train = GestureDataset(frame_folders, file_train)
ds_test = GestureDataset(frame_folders, file_test)

transform = transforms.Compose([
    transforms.ToTensor()  # Converts to FloatTensor and scales to [0.0, 1.0]
    # ... (other transforms, if necessary)
])

ds_train = GestureDataset(frame_folders, file_train, transform, sample_duration=15)
ds_test = GestureDataset(frame_folders, file_test, transform, sample_duration=15)

# train_dataloader = DataLoader(ds_train, batch_size=2, shuffle=True)
# test_dataloader = DataLoader(ds_test, batch_size=2)

model = get_resnet_transformer(feature_size=6, nhead=2, nhid=8, nlayers=1, num_classes=14)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

solver = Solver(model, ds_train, ds_test, criterion, optimizer, scheduler, device, True)

num_epochs = 1
solver.train(num_epochs)

KeyboardInterrupt: 

: 

In [3]:
solver.test()

Accuracy: 31.614906832298136%
